In [20]:
import fileinput
import json
import ast
from collections import Counter, defaultdict
import pickle
import re
from fuzzywuzzy import fuzz

In [2]:
with open('utils/conf_acr_dict.pkl', 'rb') as f:
    conf_acr = pickle.load(f)
    
with open('utils/conf_title_dict.pkl', 'rb') as f:
    conf_title = pickle.load(f)
    
with open('utils/jour_title_dict.pkl', 'rb') as f:
    jour_title = pickle.load(f)

In [3]:
re_search = '\b'+'NIPS'+'\b'
x = re.search(r'\bNIPS\b', 'MLINI@NIPS')
x

<_sre.SRE_Match object; span=(6, 10), match='NIPS'>

In [4]:
re_search = '\b'+'NIPS'+'\b'
x = re.search(r'\b'+'NIPS'+r'\b', 'MLINI@NIPS')
x

<_sre.SRE_Match object; span=(6, 10), match='NIPS'>

In [10]:
counter = 0

network_dict = {}
edge_dict = {}

# relevant_venue_fields = ['_id', 'raw', 'type', 'name_d', 'sid', 't', 'name_s', 'name', 'src']
# venue_stats = {x: Counter() for x in relevant_venue_fields}

with fileinput.input("/home/singh_shruti/data/mag_aminer/citation_network/dblpv13.json") as f:
    for line in f:
        if counter == 0:
            counter += 1
            continue
        counter += 1

        if counter%500000 == 0:
            print(counter)
        
        try:
            line = line.strip()
            if line.startswith(",{"):
                line = line[1:]
            try:
                pdict = json.loads(line)
            except:
                pdict = ast.literal_eval(line)


            if "venue" in pdict:
                if "raw" in pdict["venue"]:
                    raw_venue_name = pdict["venue"]["raw"].lower().strip()
                    found = False

                    for conf_k, acrs in conf_acr.items():
                        for aname in acrs:
    #                         re_search = '\b'+aname.lower()+'\b'
                            x = re.search(r'\b'+aname.lower().strip()+r'\b', raw_venue_name)
                            if x:
                                if "_id" in pdict and pdict["_id"]: 
                                    if type(pdict["_id"])==str:
                                        network_dict[pdict["_id"]] = pdict
                                        network_dict[pdict["_id"]].update({"venue_key_shs": conf_k})
                                    else:
                                        print(counter, pdict["_id"])
                                found = True
                                break
                        if found:
                            break
                    if not found:
                        raw_venue_name_without_of_the = raw_venue_name.replace(" of ", " ")
                        raw_venue_name_without_of_the = raw_venue_name_without_of_the.replace(" the ", " ")
                        for conf_k, acrs in conf_title.items():
                            for aname in acrs:
                                aname_clean = aname.lower().strip().replace(" of ", " ")
                                aname_clean = aname_clean.replace(" the ", " ")
                                if raw_venue_name_without_of_the.find(aname_clean) > -1:
                                    if "_id" in pdict and pdict["_id"]: 
                                        if type(pdict["_id"])==str:
                                            network_dict[pdict["_id"]] = pdict
                                            network_dict[pdict["_id"]].update({"venue_key_shs": conf_k})
                                        else:
                                            print(counter, pdict["_id"])
                                    found = True
                                    break
                            if found:
                                break

    #                     if k in relevant_venue_fields:
    #                         venue_stats[k].update([pdict["venue"][k]])
                elif "name_d" in pdict["venue"]:
                    raw_venue_name = pdict["venue"]["name_d"].lower()
                    found = False

                    for conf_k, acrs in jour_title.items():
                        for aname in acrs:
                            if raw_venue_name == aname.lower().strip():
                                if "_id" in pdict and pdict["_id"]: 
                                    if type(pdict["_id"])==str:
                                        network_dict[pdict["_id"]] = pdict
                                        network_dict[pdict["_id"]].update({"venue_key_shs": conf_k})
                                    else:
                                        print(counter, pdict["_id"])
                                found = True
                                break
                        if found:
                            break
        except Exception as ex:
            print(counter, ex)

23360 'NoneType' object has no attribute 'lower'
55666 'NoneType' object has no attribute 'lower'
166073 'NoneType' object has no attribute 'lower'
178102 'NoneType' object has no attribute 'lower'
245849 'NoneType' object has no attribute 'lower'
252246 'NoneType' object has no attribute 'lower'
293005 'NoneType' object has no attribute 'lower'
299016 'NoneType' object has no attribute 'lower'
353570 'NoneType' object has no attribute 'lower'
356247 'NoneType' object has no attribute 'lower'
381899 'NoneType' object has no attribute 'lower'
500000
563135 'NoneType' object has no attribute 'lower'
629600 'NoneType' object has no attribute 'lower'
794186 'NoneType' object has no attribute 'lower'
846895 'NoneType' object has no attribute 'lower'
867795 'NoneType' object has no attribute 'lower'
897671 'NoneType' object has no attribute 'lower'
907247 'NoneType' object has no attribute 'lower'
943031 'NoneType' object has no attribute 'lower'
1000000
1029643 'NoneType' object has no attr

3175192 'NoneType' object has no attribute 'lower'
3175487 'NoneType' object has no attribute 'lower'
3175623 'NoneType' object has no attribute 'lower'
3175687 'NoneType' object has no attribute 'lower'
3175747 'NoneType' object has no attribute 'lower'
3175998 'NoneType' object has no attribute 'lower'
3176345 'NoneType' object has no attribute 'lower'
3176582 'NoneType' object has no attribute 'lower'
3177843 'NoneType' object has no attribute 'lower'
3177975 'NoneType' object has no attribute 'lower'
3178210 'NoneType' object has no attribute 'lower'
3178445 'NoneType' object has no attribute 'lower'
3178684 'NoneType' object has no attribute 'lower'
3178836 'NoneType' object has no attribute 'lower'
3178886 'NoneType' object has no attribute 'lower'
3179196 'NoneType' object has no attribute 'lower'
3179210 'NoneType' object has no attribute 'lower'
3179290 'NoneType' object has no attribute 'lower'
3179520 'NoneType' object has no attribute 'lower'
3180691 'NoneType' object has n

In [16]:
# papers in A/A* conferences and journals
len(network_dict)

308625

In [18]:
# Count various keys present in the extracted papers

key_counter_across_all_papers = Counter()

for k,v in network_dict.items():
    key_counter_across_all_papers.update(list(v.keys()))

key_counter_across_all_papers

Counter({'_id': 308625,
         'abstract': 259031,
         'authors': 307629,
         'doi': 295262,
         'fos': 294399,
         'isbn': 257768,
         'issn': 250813,
         'issue': 275934,
         'keywords': 207900,
         'lang': 265196,
         'n_citation': 282957,
         'page_end': 303954,
         'page_start': 304545,
         'pdf': 269058,
         'references': 240972,
         'title': 308624,
         'url': 303933,
         'venue': 308625,
         'venue_key_shs': 308625,
         'volume': 276927,
         'year': 308624})

In [21]:
final_venue_stats = defaultdict(int)

for k, v in network_dict.items():
    final_venue_stats[v['venue_key_shs']] += 1
    
print(final_venue_stats)

defaultdict(<class 'int'>, {83: 2919, 7: 2416, 24: 6046, 34: 1463, 16: 11286, 25: 5534, 2: 9531, 43: 16426, 3: 5755, 0: 14849, 41: 1740, 45: 7490, 44: 2425, 76: 2347, 56: 937, 68: 513, 31: 8347, 39: 2836, 32: 2029, 70: 1545, 13: 4902, 4: 17787, 27: 4853, 71: 1625, 20: 1062, 14: 7151, 55: 2550, 51: 15509, 46: 5551, 19: 3978, 12: 4670, 8: 2382, 5: 1390, 38: 2063, 37: 6572, 50: 1890, 57: 6525, 40: 6477, 72: 422, 26: 2278, 47: 9658, 11: 5392, 78: 1413, 53: 925, 6: 12875, 18: 2737, 66: 1240, 30: 6103, 35: 536, 61: 612, 75: 3878, 52: 4672, 49: 1227, 60: 299, 54: 1476, 36: 1243, 82: 1534, 28: 1254, 22: 2330, 23: 1590, 1: 5296, 17: 518, 58: 2439, 29: 2600, 10: 1005, 64: 444, 62: 216, 9: 271, 65: 4514, 69: 530, 63: 2230, 77: 1375, 74: 410, 79: 714, 81: 482, 21: 6091, 59: 3144, 67: 1023, 42: 116, 33: 7238, 73: 379, 87: 6, 80: 433, 89: 4, 88: 33, 93: 4, 94: 27, 124: 2, 91: 3, 102: 1, 106: 2, 92: 1, 130: 1, 101: 2, 125: 1, 119: 1, 116: 1, 129: 1, 97: 1, 107: 1})


In [22]:
sum(final_venue_stats.values())

308625

In [24]:
sorted(final_venue_stats.items(), key=lambda x: x[0])[0:10]

[(0, 14849),
 (1, 5296),
 (2, 9531),
 (3, 5755),
 (4, 17787),
 (5, 1390),
 (6, 12875),
 (7, 2416),
 (8, 2382),
 (9, 271)]

In [62]:
conf_papers_count = 0
journal_papers_count = 0
confcount = 0
journcount = 0

for k, v in final_venue_stats.items():
    if k > 83:
        journcount += 1
        journal_papers_count += v
    else:
        confcount += 1
        conf_papers_count += v

In [63]:
conf_papers_count, journal_papers_count

(308533, 92)

In [64]:
confcount, journcount

(82, 18)

In [27]:
nodes = set()

for k,v in network_dict.items():
    if 'references' in v:
        nodes.update([k])
        for nb in v['references']:
            nodes.update([nb])

In [28]:
len(nodes)

743370

In [29]:
edges = defaultdict(list)

for k in nodes:
    if k in network_dict and 'references' in network_dict[k]:
        for i in network_dict[k]['references']:
            edges[k].append(i)

In [30]:
s = 0
for p in edges:
    s += len(edges[p])
    

In [31]:
s

3337053

In [11]:
type(pdict["_id"])

str

In [12]:
pdict["_id"]

'6087e112e4510cd7c80b912d'

In [5]:
all_keys = Counter()

for k,v in network_dict.items():
    all_keys.update(list(v.keys()))

In [15]:
all_keys

Counter({'_id': 139646,
         'abstract': 105646,
         'authors': 139245,
         'doi': 130096,
         'fos': 134800,
         'isbn': 108798,
         'issn': 109799,
         'issue': 120930,
         'keywords': 86308,
         'lang': 114815,
         'n_citation': 125952,
         'page_end': 136454,
         'page_start': 136539,
         'pdf': 126250,
         'references': 112114,
         'title': 139643,
         'url': 137014,
         'venue': 139646,
         'volume': 123132,
         'year': 139646})

In [6]:
nodes = set()


for k,v in network_dict.items():
    if 'references' in v:
        nodes.update([k])
        for nb in v['references']:
            nodes.update([nb])

In [7]:
len(nodes)

746776

In [9]:
from collections import defaultdict

In [10]:
edges = defaultdict(list)

for k in nodes:
    if k in network_dict and 'references' in network_dict[k]:
        for i in network_dict[k]['references']:
            edges[k].append(i)

In [21]:
from collections import defaultdict

In [11]:
s = 0
for p in edges:
    s += len(edges[p])

In [12]:
s

3345041

# Old data stats
nodes 360 thousand
edges 1 million 664 thousand
1762 ccomps
354589 - 355 thousand 

# New data stats

In [32]:
import networkx as nx
G = nx.Graph()

In [33]:
G.add_nodes_from(list(nodes))

In [34]:
edges_list = []
for i in edges:
    for k in edges[i]:
        edges_list.append((i, k))

In [35]:
G.add_edges_from(edges_list)

In [36]:
cc = list(nx.connected_components(G))

In [43]:
len_cc = []
for x in cc:
    len_cc.append(len(x))

In [46]:
sum(len_cc)

743370

In [47]:
scc = sorted(len_cc, reverse=True)

In [49]:
len(cc), len(cc[0]), scc[0:5]

(3178, 733701, [733701, 31, 28, 20, 19])

In [50]:
from networkx.algorithms import community

In [51]:
nx.info(G)

'Name: \nType: Graph\nNumber of nodes: 743370\nNumber of edges: 3329496\nAverage degree:   8.9578'

## Directed Graph Statistics

In [56]:
DG = nx.DiGraph(directed=True)

In [57]:
DG.add_nodes_from(list(nodes))

In [58]:
DG.add_edges_from(edges_list)

In [59]:
nx.info(DG)

'Name: \nType: DiGraph\nNumber of nodes: 743370\nNumber of edges: 3333025\nAverage in degree:   4.4837\nAverage out degree:   4.4837'

In [61]:
communities = community.greedy_modularity_communities(DG)

IndexError: list index out of range

# Save network dictionary

In [66]:
with open('data/network_dict.pkl', 'wb') as f:
    pickle.dump(network_dict, f)